In [1]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset,SVD
from surprise.model_selection import cross_validate
from surprise import KNNBasic
import math
import random

In [2]:
df = pd.read_csv("Data\sp500_companies.csv")

df

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
0,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software - Infrastructure,423.08,3144461582336,1.251820e+11,0.170,Redmond,WA,United States,221000.0,Microsoft Corporation develops and supports so...,0.063479
1,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,189.72,2909185507328,1.296290e+11,-0.043,Cupertino,CA,United States,150000.0,"Apple Inc. designs, manufactures, and markets ...",0.058729
2,NMS,NVDA,NVIDIA Corporation,NVIDIA Corporation,Technology,Semiconductors,946.30,2365749985280,3.448000e+10,2.653,Santa Clara,CA,United States,29600.0,NVIDIA Corporation provides graphics and compu...,0.047758
3,NMS,GOOGL,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,172.51,2126875721728,1.097230e+11,0.154,Mountain View,CA,United States,180895.0,Alphabet Inc. offers various products and plat...,0.042936
4,NMS,GOOG,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,173.88,2123944296448,1.097230e+11,0.154,Mountain View,CA,United States,180895.0,Alphabet Inc. offers various products and plat...,0.042877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,NMS,ETSY,"Etsy, Inc.","Etsy, Inc.",Consumer Cyclical,Internet Retail,64.57,7550363648,4.262970e+08,0.008,Brooklyn,NY,United States,2420.0,"Etsy, Inc., together with its subsidiaries, op...",0.000152
499,NYQ,RHI,Robert Half Inc.,Robert Half Inc.,Industrials,Staffing & Employment Services,71.16,7474362368,4.228020e+08,-0.140,Menlo Park,CA,United States,15000.0,Robert Half Inc. provides talent solutions and...,0.000151
500,NYQ,IVZ,Invesco Ltd,Invesco Ltd.,Financial Services,Asset Management,16.31,7336743424,9.988000e+08,0.040,Atlanta,GA,United States,8527.0,Invesco Ltd. is a publicly owned investment ma...,0.000148
501,NYQ,CMA,Comerica Incorporated,Comerica Incorporated,Financial Services,Banks - Regional,55.18,7316150784,NaN,-0.198,Dallas,TX,United States,7619.0,"Comerica Incorporated, through its subsidiarie...",0.000148


In [3]:

details_arr = []

for i  in range(0,100):
    customer_id = i
    num_shares = random.randint(20,30)
    random_num_list = random.sample(range(0,502),num_shares)
    for j in range(0,num_shares):
        index = random_num_list[j]
        amount = random.randint(500,5000)
        details_arr.append([customer_id,df["Symbol"][index],amount])
    

details_df = pd.DataFrame(details_arr, columns=["Customer_Id","Company_Ticker","Amount"])


details_df.head(50)




,Customer_Id,Company_Ticker,Amount
0,0,DE,672
1,0,ALB,4469
2,0,ADI,1935
3,0,TTWO,3536
4,0,CEG,538
5,0,MKTX,2307
6,0,SYF,2086
7,0,CAH,3568
8,0,BX,1734
9,0,CBOE,2744


In [4]:
details_df.to_csv('Data_Customer_Portfolio.csv', index=False)

In [5]:
df_customer0 = details_df[(details_df["Customer_Id"] == 0)]

df_customer0

,Customer_Id,Company_Ticker,Amount
0,0,DE,672
1,0,ALB,4469
2,0,ADI,1935
3,0,TTWO,3536
4,0,CEG,538
5,0,MKTX,2307
6,0,SYF,2086
7,0,CAH,3568
8,0,BX,1734
9,0,CBOE,2744
